In [1]:
import os
import numpy as np
from skimage.io import imread
from skimage.filters import threshold_otsu
import keras
from keras import optimizers
from keras.layers import *
from keras.models import Model
from keras.utils import to_categorical



Using TensorFlow backend.


In [2]:
letters = [
            '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D',
            'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T',
            'U', 'V', 'W', 'X', 'Y', 'Z'
        ]

def read_training_data(training_directory):
    training_data=[]
    for each_letter in letters:
        for each in range(10):
            image_path = os.path.join(training_directory, each_letter, each_letter + '_' + str(each) + '.jpg')
            # read each image of each character
            img_details = imread(image_path, as_gray=True)
            # converts each character image to binary image
            binary_image = img_details < threshold_otsu(img_details)
            # 20*20 image becomes 1*400
            # in machine learning terms that's 400 features
            flat_bin_image = binary_image.reshape(-1)
            training_data.append([flat_bin_image, each_letter])

    return (np.array(training_data))

In [3]:
# current_dir = os.path.dirname(os.path.realpath(__file__))
# training_dataset_dir = os.path.join(current_dir, 'train')
print('reading data')
training_dataset_dir = './train20X20'
training_data = read_training_data(training_dataset_dir)
print(training_data.shape)
print('reading data completed')

reading data
(340, 2)
reading data completed


In [4]:
#Randomizing the data for better training of the model 
import random 
random.shuffle(training_data)

X=[]
y=[]
for features, labels in training_data:
    X.append(features)
    y.append(labels)

X=np.array(X)
print(X.shape)
print(y)

(340, 400)
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '1', '1', '0', '0', '0', '0', '1', '2', '1', '2', '1', '0', '1', '2', '2', '1', '1', '2', '1', '1', '1', '1', '3', '0', '3', '2', '0', '2', '4', '0', '3', '1', '0', '0', '1', '2', '1', '3', '1', '5', '3', '3', '0', '2', '1', '3', '5', '3', '0', '2', '0', '3', '2', '1', '5', '2', '2', '6', '5', '1', '4', '3', '3', '3', '1', '4', '3', '7', '3', '1', '0', '1', '5', '5', '5', '8', '0', '8', '5', '5', '2', '3', '1', '7', '5', '4', '9', '0', '2', '7', '7', '4', '5', '0', '4', '5', '3', 'A', '0', '5', '2', '7', '3', 'B', '1', '1', '6', '0', '6', '5', '4', 'A', '3', '7', '1', '7', 'D', 'C', 'A', '4', '0', 'A', '6', '0', 'C', '7', '5', '6', 'C', '1', '2', 'E', 'D', '8', '7', '9', '2', '3', '4', '4', '2', '9', '3', '0', '9', 'E', '4', '3', '5', '5', '1', 'F', 'F', 'F', 'C', '4', 'F', 'H', 'D', '3', 'G', '5', '4', '9', 'E', '9', 'H', '3', 'H', '7', 'F', 'E', 'E', '5', '6', '1', '1', '4', '8', 'B', 'K', 'D'

In [5]:
# preprocessing data
#encodeing string labels to integers
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
#print(integer_encoded)
labels_encoded=to_categorical(integer_encoded, num_classes=34) #since it is multiclass classification it converts 34 class vectors to binary class matrix
#print(labels_encoded)

X=X/255

In [6]:
print('=======================training model============================')      #keras module used 
Inp=Input((400,))
x=(Dense(300, activation='tanh'))(Inp)
x=(Dense(200, activation='tanh'))(x)
x=(Dense(100, activation='tanh'))(x)
x=(Dense(100, activation='tanh'))(x)
x=(Dense(100, activation='tanh'))(x)
x=(Dense(100, activation='tanh'))(x)
output= (Dense(34, activation='softmax'))(x)

model=Model(Inp, output)
model.summary()

W0930 18:55:15.159904 4701914560 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0930 18:55:15.178949 4701914560 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0930 18:55:15.186120 4701914560 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



=======================training model============================
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 400)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               120300    
_________________________________________________________________
dense_2 (Dense)              (None, 200)               60200     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_6 (D

In [7]:
learning_rate=0.001
epochs=30
batch_size=20
adam=optimizers.Adam(lr=learning_rate)


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, labels_encoded, batch_size=batch_size, epochs=epochs, validation_split=0.2)

W0930 18:55:24.761082 4701914560 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0930 18:55:24.793860 4701914560 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0930 18:55:24.874943 4701914560 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0930 18:55:24.933285 4701914560 deprecation_wrapper.py:119] From /Library/Frameworks

Train on 272 samples, validate on 68 samples
Epoch 1/30
272/272 [==============================] - 1s 2ms/step - loss: 3.2932 - acc: 0.1360 - val_loss: 3.8275 - val_acc: 0.0147
Epoch 2/30
272/272 [==============================] - 0s 322us/step - loss: 2.8590 - acc: 0.2316 - val_loss: 3.7618 - val_acc: 0.0441
Epoch 3/30
272/272 [==============================] - 0s 269us/step - loss: 2.4161 - acc: 0.2831 - val_loss: 3.2883 - val_acc: 0.1324
Epoch 4/30
272/272 [==============================] - 0s 280us/step - loss: 2.0784 - acc: 0.4007 - val_loss: 3.3161 - val_acc: 0.1471
Epoch 5/30
272/272 [==============================] - 0s 262us/step - loss: 1.8018 - acc: 0.5074 - val_loss: 2.9675 - val_acc: 0.2647
Epoch 6/30
272/272 [==============================] - 0s 282us/step - loss: 1.4909 - acc: 0.5882 - val_loss: 2.8485 - val_acc: 0.2941
Epoch 7/30
272/272 [==============================] - 0s 258us/step - loss: 1.2779 - acc: 0.6397 - val_loss: 2.6220 - val_acc: 0.3382
Epoch 8/30
272/272 